# This is the Analysis of the Churn Dataset

In [1]:
#import the libraries
import pandas as pd
import sqlite3
import warnings
warnings.filterwarnings("ignore")

In [2]:
#read all csv files with pandas
customer = pd.read_csv("customer.csv")
customer.head(7)

,CustomerID,Gender,SeniorCitizen,Partner,Dependents
0,3668-QPYBK,Male,No,No,No
1,9237-HQITU,Female,No,No,Yes
2,9305-CDSKC,Female,No,No,Yes
3,7892-POOKP,Female,No,Yes,Yes
4,0280-XJGEX,Male,No,No,Yes
5,4190-MFLUW,Female,No,Yes,No
6,8779-QRDMV,Male,Yes,No,No


In [3]:
cust_account = pd.read_csv("cust_account.csv")
cust_account.head(7)

,Account_id,Tenure,Contract,PaymentMethod,PaperlessBilling,MonthlyCharges,TotalCharges
0,3668-QPYBK,2,Month-to-month,Mailed check,Yes,53.85,108.15
1,9237-HQITU,2,Month-to-month,Electronic check,Yes,70.70,151.65
2,9305-CDSKC,8,Month-to-month,Electronic check,Yes,99.65,820.5
3,7892-POOKP,28,Month-to-month,Electronic check,Yes,104.80,3046.05
4,0280-XJGEX,49,Month-to-month,Bank transfer (automatic),Yes,103.70,5036.3
5,4190-MFLUW,10,Month-to-month,Credit card (automatic),No,55.20,528.35
6,8779-QRDMV,1,Month-to-month,Electronic check,Yes,39.65,39.65


In [4]:
cust_churn = pd.read_csv("cust_churn.csv")
cust_churn.head(7)

,Id,Churn
0,3668-QPYBK,Yes
1,9237-HQITU,Yes
2,9305-CDSKC,Yes
3,7892-POOKP,Yes
4,0280-XJGEX,Yes
5,4190-MFLUW,Yes
6,8779-QRDMV,Yes


In [5]:
cust_loc = pd.read_csv("cust_loc.csv")
cust_loc.head(7)

,Cust_ID,State,Latitude,Longitude,ZipCode
0,3668-QPYBK,California,33.964131,-118.272783,90003
1,9237-HQITU,California,34.059281,-118.307420,90005
2,9305-CDSKC,California,34.048013,-118.293953,90006
3,7892-POOKP,California,34.062125,-118.315709,90010
4,0280-XJGEX,California,34.039224,-118.266293,90015
5,4190-MFLUW,California,34.066367,-118.309868,90020
6,8779-QRDMV,California,34.023810,-118.156582,90022


In [7]:
cust_services = pd.read_csv("cust_services.csv")
cust_services.head(7)

,Cust_ID,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
0,3668-QPYBK,Yes,No,DSL,Yes,Yes,No,No,No,No
1,9237-HQITU,Yes,No,Fiber optic,No,No,No,No,No,No
2,9305-CDSKC,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes
3,7892-POOKP,Yes,Yes,Fiber optic,No,No,Yes,Yes,Yes,Yes
4,0280-XJGEX,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes
5,4190-MFLUW,Yes,No,DSL,No,No,Yes,Yes,No,No
6,8779-QRDMV,No,No phone service,DSL,No,No,Yes,No,No,Yes


### Create churn database and tables

In [9]:
#database called churn.db
conn = sqlite3.connect('telecomchurn.db')
cur = conn.cursor()

In [10]:
# create cust_account table
cust_account.to_sql("cust_account", conn, if_exists="replace", index=False)

In [11]:
# create cust_churn table
cust_churn.to_sql("cust_churn", conn, if_exists="replace", index=False)

In [12]:
# create cust_loc table
cust_loc.to_sql("cust_loc", conn, if_exists="replace", index=False)

In [13]:
# create cust_service table
cust_services.to_sql("cust_services", conn, if_exists="replace", index=False)

In [14]:
# create customer table
customer.to_sql("customer", conn, if_exists="replace", index=False)

In [15]:
#Get information about our tables

def table_info(conn,cursor):
    """
    prints out all of the columns of every table in the DB
    
    conn: database connection object
    cursor: cursor object
    """
    
    tables = cur.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()
    for table_name in tables:
        table_name = table_name[0]
        table = pd.read_sql_query("SELECT * from {} LIMIT 0".format(table_name), conn)
        print(table_name)
        for col in table.columns:
            print("\t "+ col)
            print()

In [18]:
table_info(conn,cur)

cust_account
	 Account_id

	 Tenure

	 Contract

	 PaymentMethod

	 PaperlessBilling

	 MonthlyCharges

	 TotalCharges

cust_churn
	 Id

	 Churn

cust_loc
	 CustomerID

	 State

	 Latitude

	 Longitude

	 ZipCode

cust_services
	 Cust_ID

	 PhoneService

	 MultipleLines

	 InternetService

	 OnlineSecurity

	 OnlineBackup

	 DeviceProtection

	 TechSupport

	 StreamingTV

	 StreamingMovies

customer
	 CustomerID

	 Gender

	 SeniorCitizen

	 Partner

	 Dependents



In [17]:
#Let's join all the table into one big table

#rename cust_ID to CustomerID

cur.execute(
"""
ALTER TABLE cust_loc
RENAME COLUMN Cust_ID TO CustomerID;
""")

In [20]:
cur.execute(

"""
SELECT * FROM customer
INNER JOIN cust_loc
USING (CustomerID)
""")
cur.fetchone()

('3668-QPYBK',
 'Male',
 'No',
 'No',
 'No',
 'California',
 33.964131,
 -118.272783,
 90003)

In [26]:
cur.execute("""ALTER TABLE cust_services RENAME COLUMN Cust_ID to CustomerID;""") 

In [21]:
cust_loc.head()

,Cust_ID,State,Latitude,Longitude,ZipCode
0,3668-QPYBK,California,33.964131,-118.272783,90003
1,9237-HQITU,California,34.059281,-118.307420,90005
2,9305-CDSKC,California,34.048013,-118.293953,90006
3,7892-POOKP,California,34.062125,-118.315709,90010
4,0280-XJGEX,California,34.039224,-118.266293,90015


In [22]:
cust_loc = cust_loc.rename(columns={"Cust_ID":"CustomerID"})
cust_loc.head()

,CustomerID,State,Latitude,Longitude,ZipCode
0,3668-QPYBK,California,33.964131,-118.272783,90003
1,9237-HQITU,California,34.059281,-118.307420,90005
2,9305-CDSKC,California,34.048013,-118.293953,90006
3,7892-POOKP,California,34.062125,-118.315709,90010
4,0280-XJGEX,California,34.039224,-118.266293,90015


In [23]:
customer.head()

,CustomerID,Gender,SeniorCitizen,Partner,Dependents
0,3668-QPYBK,Male,No,No,No
1,9237-HQITU,Female,No,No,Yes
2,9305-CDSKC,Female,No,No,Yes
3,7892-POOKP,Female,No,Yes,Yes
4,0280-XJGEX,Male,No,No,Yes


In [24]:
#Join the tables using the concat function
pd.concat([customer,cust_loc], join = "inner", axis=1)

,CustomerID,Gender,SeniorCitizen,Partner,Dependents,CustomerID,State,Latitude,Longitude,ZipCode
0,3668-QPYBK,Male,No,No,No,3668-QPYBK,California,33.964131,-118.272783,90003
1,9237-HQITU,Female,No,No,Yes,9237-HQITU,California,34.059281,-118.307420,90005
2,9305-CDSKC,Female,No,No,Yes,9305-CDSKC,California,34.048013,-118.293953,90006
3,7892-POOKP,Female,No,Yes,Yes,7892-POOKP,California,34.062125,-118.315709,90010
4,0280-XJGEX,Male,No,No,Yes,0280-XJGEX,California,34.039224,-118.266293,90015
...,...,...,...,...,...,...,...,...,...,...
7038,2569-WGERO,Female,No,No,No,2569-WGERO,California,34.341737,-116.539416,92285
7039,6840-RESVB,Male,No,Yes,Yes,6840-RESVB,California,34.667815,-117.536183,92301
7040,2234-XADUH,Female,No,Yes,Yes,2234-XADUH,California,34.559882,-115.637164,92304
7041,4801-JZAZL,Female,No,Yes,Yes,4801-JZAZL,California,34.167800,-116.864330,92305


In [30]:
cust_loc.head()

,CustomerID,State,Latitude,Longitude,ZipCode
0,3668-QPYBK,California,33.964131,-118.272783,90003
1,9237-HQITU,California,34.059281,-118.307420,90005
2,9305-CDSKC,California,34.048013,-118.293953,90006
3,7892-POOKP,California,34.062125,-118.315709,90010
4,0280-XJGEX,California,34.039224,-118.266293,90015


In [31]:
#Let's use the same pandas approach to rename columns
cust_services = cust_services.rename(columns={"Cust_ID":"CustomerID"})
cust_account = cust_account.rename(columns={"Account_id":"CustomerID"})
cust_churn = cust_churn.rename(columns={"id":"CustomerID"})


In [33]:
#put all dataframes in a list

dfList = [customer, cust_account, cust_churn, cust_loc, cust_services]

In [34]:
#concat all the dataframe into one
all_churn = pd.concat(dfList, join="inner", axis=1)

In [35]:
all_churn.head()

,CustomerID,Gender,SeniorCitizen,Partner,Dependents,CustomerID,Tenure,Contract,PaymentMethod,PaperlessBilling,...,CustomerID,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
0,3668-QPYBK,Male,No,No,No,3668-QPYBK,2,Month-to-month,Mailed check,Yes,...,3668-QPYBK,Yes,No,DSL,Yes,Yes,No,No,No,No
1,9237-HQITU,Female,No,No,Yes,9237-HQITU,2,Month-to-month,Electronic check,Yes,...,9237-HQITU,Yes,No,Fiber optic,No,No,No,No,No,No
2,9305-CDSKC,Female,No,No,Yes,9305-CDSKC,8,Month-to-month,Electronic check,Yes,...,9305-CDSKC,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes
3,7892-POOKP,Female,No,Yes,Yes,7892-POOKP,28,Month-to-month,Electronic check,Yes,...,7892-POOKP,Yes,Yes,Fiber optic,No,No,Yes,Yes,Yes,Yes
4,0280-XJGEX,Male,No,No,Yes,0280-XJGEX,49,Month-to-month,Bank transfer (automatic),Yes,...,0280-XJGEX,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes


In [37]:
all_churn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 29 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   CustomerID        7043 non-null   object 
 1   Gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   object 
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   CustomerID        7043 non-null   object 
 6   Tenure            7043 non-null   int64  
 7   Contract          7043 non-null   object 
 8   PaymentMethod     7043 non-null   object 
 9   PaperlessBilling  7043 non-null   object 
 10  MonthlyCharges    7043 non-null   float64
 11  TotalCharges      7043 non-null   object 
 12  Id                7043 non-null   object 
 13  Churn             7043 non-null   object 
 14  CustomerID        7043 non-null   object 
 15  State             7043 non-null   object 
 16  Latitude          7043 non-null   float64


In [38]:
all_churn.columns.duplicated()

array([False, False, False, False, False,  True, False, False, False,
       False, False, False, False, False,  True, False, False, False,
       False,  True, False, False, False, False, False, False, False,
       False, False])

In [39]:
#Get rid of all the duplicate columns
all_churn = all_churn.loc[:,~all_churn.columns.duplicated()]

In [40]:
all_churn.head()

,CustomerID,Gender,SeniorCitizen,Partner,Dependents,Tenure,Contract,PaymentMethod,PaperlessBilling,MonthlyCharges,...,ZipCode,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
0,3668-QPYBK,Male,No,No,No,2,Month-to-month,Mailed check,Yes,53.85,...,90003,Yes,No,DSL,Yes,Yes,No,No,No,No
1,9237-HQITU,Female,No,No,Yes,2,Month-to-month,Electronic check,Yes,70.70,...,90005,Yes,No,Fiber optic,No,No,No,No,No,No
2,9305-CDSKC,Female,No,No,Yes,8,Month-to-month,Electronic check,Yes,99.65,...,90006,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes
3,7892-POOKP,Female,No,Yes,Yes,28,Month-to-month,Electronic check,Yes,104.80,...,90010,Yes,Yes,Fiber optic,No,No,Yes,Yes,Yes,Yes
4,0280-XJGEX,Male,No,No,Yes,49,Month-to-month,Bank transfer (automatic),Yes,103.70,...,90015,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes


In [41]:
all_churn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 26 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   CustomerID        7043 non-null   object 
 1   Gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   object 
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   Tenure            7043 non-null   int64  
 6   Contract          7043 non-null   object 
 7   PaymentMethod     7043 non-null   object 
 8   PaperlessBilling  7043 non-null   object 
 9   MonthlyCharges    7043 non-null   float64
 10  TotalCharges      7043 non-null   object 
 11  Id                7043 non-null   object 
 12  Churn             7043 non-null   object 
 13  State             7043 non-null   object 
 14  Latitude          7043 non-null   float64
 15  Longitude         7043 non-null   float64
 16  ZipCode           7043 non-null   int64  


In [42]:
#Let's change TotalCharges to Numeric
all_churn["TotalCharges"] = pd.to_numeric(all_churn["TotalCharges"],errors = "coerce")

In [43]:
all_churn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 26 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   CustomerID        7043 non-null   object 
 1   Gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   object 
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   Tenure            7043 non-null   int64  
 6   Contract          7043 non-null   object 
 7   PaymentMethod     7043 non-null   object 
 8   PaperlessBilling  7043 non-null   object 
 9   MonthlyCharges    7043 non-null   float64
 10  TotalCharges      7032 non-null   float64
 11  Id                7043 non-null   object 
 12  Churn             7043 non-null   object 
 13  State             7043 non-null   object 
 14  Latitude          7043 non-null   float64
 15  Longitude         7043 non-null   float64
 16  ZipCode           7043 non-null   int64  


In [44]:
all_churn.isnull().sum(axis=0)

CustomerID           0
Gender               0
SeniorCitizen        0
Partner              0
Dependents           0
Tenure               0
Contract             0
PaymentMethod        0
PaperlessBilling     0
MonthlyCharges       0
TotalCharges        11
Id                   0
Churn                0
State                0
Latitude             0
Longitude            0
ZipCode              0
PhoneService         0
MultipleLines        0
InternetService      0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
dtype: int64

In [45]:
#drop the missing values
all_churn = all_churn.dropna()

In [46]:
all_churn.isnull().sum(axis=0)

CustomerID          0
Gender              0
SeniorCitizen       0
Partner             0
Dependents          0
Tenure              0
Contract            0
PaymentMethod       0
PaperlessBilling    0
MonthlyCharges      0
TotalCharges        0
Id                  0
Churn               0
State               0
Latitude            0
Longitude           0
ZipCode             0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
dtype: int64

In [47]:
all_churn.head()

,CustomerID,Gender,SeniorCitizen,Partner,Dependents,Tenure,Contract,PaymentMethod,PaperlessBilling,MonthlyCharges,...,ZipCode,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
0,3668-QPYBK,Male,No,No,No,2,Month-to-month,Mailed check,Yes,53.85,...,90003,Yes,No,DSL,Yes,Yes,No,No,No,No
1,9237-HQITU,Female,No,No,Yes,2,Month-to-month,Electronic check,Yes,70.70,...,90005,Yes,No,Fiber optic,No,No,No,No,No,No
2,9305-CDSKC,Female,No,No,Yes,8,Month-to-month,Electronic check,Yes,99.65,...,90006,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes
3,7892-POOKP,Female,No,Yes,Yes,28,Month-to-month,Electronic check,Yes,104.80,...,90010,Yes,Yes,Fiber optic,No,No,Yes,Yes,Yes,Yes
4,0280-XJGEX,Male,No,No,Yes,49,Month-to-month,Bank transfer (automatic),Yes,103.70,...,90015,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes


In [50]:
all_churn.shape

(7032, 26)

In [51]:
churn_all=pd.concat([customer,cust_loc,cust_account,cust_churn,cust_services],join="inner",axis=1)

churn_all=churn_all.iloc[:,~churn_all.columns.duplicated()]

churn_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 26 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   CustomerID        7043 non-null   object 
 1   Gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   object 
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   State             7043 non-null   object 
 6   Latitude          7043 non-null   float64
 7   Longitude         7043 non-null   float64
 8   ZipCode           7043 non-null   int64  
 9   Tenure            7043 non-null   int64  
 10  Contract          7043 non-null   object 
 11  PaymentMethod     7043 non-null   object 
 12  PaperlessBilling  7043 non-null   object 
 13  MonthlyCharges    7043 non-null   float64
 14  TotalCharges      7043 non-null   object 
 15  Id                7043 non-null   object 
 16  Churn             7043 non-null   object 


In [49]:
#save all_churn back to a table in the telecomchurn.db database
all_churn.to_sql("all_churn", conn, if_exists="replace", index=False)

In [52]:
#sql codes with pandas
pd.read_sql("select * from all_churn", conn)

,CustomerID,Gender,SeniorCitizen,Partner,Dependents,Tenure,Contract,PaymentMethod,PaperlessBilling,MonthlyCharges,...,ZipCode,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
0,3668-QPYBK,Male,No,No,No,2,Month-to-month,Mailed check,Yes,53.85,...,90003,Yes,No,DSL,Yes,Yes,No,No,No,No
1,9237-HQITU,Female,No,No,Yes,2,Month-to-month,Electronic check,Yes,70.70,...,90005,Yes,No,Fiber optic,No,No,No,No,No,No
2,9305-CDSKC,Female,No,No,Yes,8,Month-to-month,Electronic check,Yes,99.65,...,90006,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes
3,7892-POOKP,Female,No,Yes,Yes,28,Month-to-month,Electronic check,Yes,104.80,...,90010,Yes,Yes,Fiber optic,No,No,Yes,Yes,Yes,Yes
4,0280-XJGEX,Male,No,No,Yes,49,Month-to-month,Bank transfer (automatic),Yes,103.70,...,90015,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7027,2569-WGERO,Female,No,No,No,72,Two year,Bank transfer (automatic),Yes,21.15,...,92285,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service
7028,6840-RESVB,Male,No,Yes,Yes,24,One year,Mailed check,Yes,84.80,...,92301,Yes,Yes,DSL,Yes,No,Yes,Yes,Yes,Yes
7029,2234-XADUH,Female,No,Yes,Yes,72,One year,Credit card (automatic),Yes,103.20,...,92304,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes
7030,4801-JZAZL,Female,No,Yes,Yes,11,Month-to-month,Electronic check,Yes,29.60,...,92305,No,No phone service,DSL,Yes,No,No,No,No,No


In [53]:
#filter for the records of customers who have not left the company

churn_df = pd.read_sql("select * from all_churn where Churn = 'Yes'", conn)

In [54]:
churn_df.head()

,CustomerID,Gender,SeniorCitizen,Partner,Dependents,Tenure,Contract,PaymentMethod,PaperlessBilling,MonthlyCharges,...,ZipCode,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
0,3668-QPYBK,Male,No,No,No,2,Month-to-month,Mailed check,Yes,53.85,...,90003,Yes,No,DSL,Yes,Yes,No,No,No,No
1,9237-HQITU,Female,No,No,Yes,2,Month-to-month,Electronic check,Yes,70.70,...,90005,Yes,No,Fiber optic,No,No,No,No,No,No
2,9305-CDSKC,Female,No,No,Yes,8,Month-to-month,Electronic check,Yes,99.65,...,90006,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes
3,7892-POOKP,Female,No,Yes,Yes,28,Month-to-month,Electronic check,Yes,104.80,...,90010,Yes,Yes,Fiber optic,No,No,Yes,Yes,Yes,Yes
4,0280-XJGEX,Male,No,No,Yes,49,Month-to-month,Bank transfer (automatic),Yes,103.70,...,90015,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes


In [55]:
pd.__version__

'1.0.3'

In [ ]:
!pip install pandas

In [56]:
churn_all.query('Churn == "Yes"')

,CustomerID,Gender,SeniorCitizen,Partner,Dependents,State,Latitude,Longitude,ZipCode,Tenure,...,Churn,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
0,3668-QPYBK,Male,No,No,No,California,33.964131,-118.272783,90003,2,...,Yes,Yes,No,DSL,Yes,Yes,No,No,No,No
1,9237-HQITU,Female,No,No,Yes,California,34.059281,-118.307420,90005,2,...,Yes,Yes,No,Fiber optic,No,No,No,No,No,No
2,9305-CDSKC,Female,No,No,Yes,California,34.048013,-118.293953,90006,8,...,Yes,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes
3,7892-POOKP,Female,No,Yes,Yes,California,34.062125,-118.315709,90010,28,...,Yes,Yes,Yes,Fiber optic,No,No,Yes,Yes,Yes,Yes
4,0280-XJGEX,Male,No,No,Yes,California,34.039224,-118.266293,90015,49,...,Yes,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1864,1699-HPSBG,Male,No,No,No,California,33.745746,-116.514215,92264,12,...,Yes,Yes,No,DSL,No,No,No,Yes,Yes,No
1865,8775-CEBBJ,Female,No,No,No,California,32.790282,-115.689559,92273,9,...,Yes,Yes,No,DSL,No,No,No,No,No,No
1866,6894-LFHLY,Male,Yes,No,No,California,34.264124,-114.717964,92280,1,...,Yes,Yes,Yes,Fiber optic,No,No,No,No,No,No
1867,0639-TSIQW,Female,No,No,No,California,33.972293,-116.654195,92282,67,...,Yes,Yes,Yes,Fiber optic,Yes,Yes,Yes,No,Yes,No


In [57]:
#let's save all_churn table to a csv file

all_churn.to_csv('all_churn.csv')

In [ ]:
careergyms@gmail.com